In [ ]:
'''
Social network visualization
The network(...) function code is adapted from https://earlyprint.org/jupyterbook/metadata.html
'''
import re
import pandas as pd
import networkx as nx 
from pyvis.network import Network

def authors(csv):
    '''
    Extract text IDs and standardize authors' names.
    '''
    df = pd.read_csv(csv)
    authors = df['author']
    ids = df['id']
    numFiles = len(ids)
    count = 0
    dict = {}
    while count < numFiles:
        words = set(authors[count].split(';'))
        # removing unnecessary keywords
        words.discard('')
        # Removing unnecessary dates  
        newWords = []
        for w in words: 
            w = w.replace('.','')
            # w = re.sub(r'\([^)]*\)','',w)
            if re.search('printer',w):
                continue
            w = re.sub(r' ca|-|[0-9]{4}|, Sir,|, Sir|fl|, d|,  A profession of faith|, Mr','',w)
            w = re.sub(r', Earl of,  or 8|, ca|,  aut|, \?|,  Of schisme|, 16th/17th cent|, attributed name','',w)
            w = re.sub(r',   aut|, bachelor of civil law| Tragedie of Darius| Monarchick tragedies|Doomesday| Paraenesis to the Prince|,  A profession of faith','',w)
            w = w.strip('|,|, ')
            newWords.append(w)
        dict[ids[count]] = list(set(newWords))
        count += 1
    return dict 

def network(dict,metadata,title):
    '''
    Catalogs edges between nodes and outputs the visualization as a HTML file. 
    '''
    edgelist = []    
    for id in dict.keys():
        for name in dict[id]:
            edgelist.append((id, name))
    B = nx.Graph() # Create an empty graph object

    # Create lists of our two node types
    ids = []
    names = []
    for e in edgelist:
        ids.append(e[0])
        auth.append(e[1])
    # Make sure names or IDs don't repeat
    texts = list(set(ids))
    names = list(set(names))
    # Add each node list to our graph, with bipartite and group attributes to keep track of which is which
    B.add_nodes_from(texts, bipartite="text", group=1)
    B.add_nodes_from(names, bipartite=metadata, group=2)
    B.add_edges_from(edgelist)

    # Create pyvis graph
    g = Network(width=1200,height=800,notebook=True,heading='')
    for id,auth in edgelist:
        g.add_node(id, id, title=id, color='purple')
        g.add_node(auth, auth, title=auth, color='pink')    
        g.add_edge(id,auth)
    g.show(title+".html")

In [ ]:
csv = '/srv/data/CSVs/virginia.csv'
auth = authors(csv)
network(auth,r'author(s)','virginiaAuthors')